<img src="NotebookAddons/blackboard-banner.png" width="100%" />
<font face="Calibri">
<br>
<font size="5"> <b>An Introduction to Simple SAR Change Detection Methods</b></font>

<br>
<font size="4"> <b> Franz J Meyer; University of Alaska Fairbanks </b> <br>
<img src="NotebookAddons/UAFLogo_A_647.png" style="padding:5px;" width="170" align="right" /></font>

<font size="3">This notebook introduces you to a some popular change detection methods that can be applied to SAR time series data. SAR is an excellent tool for change detection. As SAR sensors are weather and illumination independent and carry their own illumination source (active sensor). Differences between repeated images are a direct indication of changes on the surface. This fact is exploited by the change detection methods introduced below. 
    
The exercise is done in the framework of *Jupyter Notebooks*. The Jupyter Notebook environment is easy to launch in any web browser for interactive data exploration with provided or new training data. Notebooks are comprised of text written in a combination of executable python code and markdown formatting including latex style mathematical equations. Another advantage of Jupyter Notebooks is that they can easily be expanded, changed, and shared with new data sets or newly available time series steps. Therefore, they provide an excellent basis for collaborative and repeatable data analysis. <br>

<b>This notebook covers the following data analysis concepts:</b>

- Time series metrics  95$^{th}$ and 5$^{th}$ percentile difference thresholding
- Time series coefficient of variation thresholding
- Log Ratio-based change detection from image pairs

</font>


</font>

<hr>
<font face="Calibri" size="5" color='rgba(200,0,0,0.2)'> <b>Important Note about JupyterHub</b> </font>
<br><br>
<font face="Calibri" size="3"> <b>Your JupyterHub server will automatically shutdown when left idle for more than 1 hour. Your notebooks will not be lost but you will have to restart their kernels and re-run them from the beginning. You will not be able to seamlessly continue running a partially run notebook.</b> </font>


<hr>
<font face="Calibri">

<font size="5"> <b> 0. Importing Relevant Python Packages </b> </font>

<font size="3">In this notebook we will use the following scientific libraries:
<ol type="1">
    <li> <b><a href="https://pandas.pydata.org/" target="_blank">Pandas</a></b> is a Python library that provides high-level data structures and a vast variety of tools for analysis. The great feature of this package is the ability to translate rather complex operations with data into one or two commands. Pandas contains many built-in methods for filtering and combining data, as well as the time-series functionality. </li>
    <li> <b><a href="https://www.gdal.org/" target="_blank">GDAL</a></b> is a software library for reading and writing raster and vector geospatial data formats. It includes a collection of programs tailored for geospatial data processing. Most modern GIS systems (such as ArcGIS or QGIS) use GDAL in the background.</li>
    <li> <b><a href="http://www.numpy.org/" target="_blank">NumPy</a></b> is one of the principal packages for scientific applications of Python. It is intended for processing large multidimensional arrays and matrices, and an extensive collection of high-level mathematical functions and implemented methods makes it possible to perform various operations with these objects. </li>
    <li> <b><a href="https://matplotlib.org/index.html" target="_blank">Matplotlib</a></b> is a low-level library for creating two-dimensional diagrams and graphs. With its help, you can build diverse charts, from histograms and scatterplots to non-Cartesian coordinates graphs. Moreover, many popular plotting libraries are designed to work in conjunction with matplotlib. </li>
    <li> The <b><a href="https://www.pydoc.io/pypi/asf-hyp3-1.1.1/index.html" target="_blank">asf-hyp3 API</a></b> provides useful functions and scripts for accessing and processing SAR data via the Alaska Satellite Facility's Hybrid Pluggable Processing Pipeline, or HyP3 (pronounced "hype"). </li>
<li><b><a href="https://www.scipy.org/about.html" target="_blank">SciPY</a></b> is a library that provides functions for numerical integration, interpolation, optimization, linear algebra and statistics. </li>

</font>

<font face="Calibri" size="3"> Our first step is to <b>import them:</b> </font>

In [ ]:
!pip install scikit-image

In [ ]:
import os # for chdir, getcwd, path.exists
import datetime # for date

import pandas as pd # for DatetimeIndex
from osgeo import gdal # for Info

import matplotlib.pyplot as plt
from matplotlib import animation
import matplotlib.patches as patches  # for Rectangle
from matplotlib import rc
import numpy as np

from IPython.display import HTML

from asf_notebook import path_exists
from asf_notebook import asf_unzip
from asf_notebook import new_directory
from asf_notebook import jupytertheme_matplotlib_format

<font face="Calibri" size="3"><b>Setup matplotlib plotting</b> inside the notebook:</font>

In [ ]:
jupytertheme_matplotlib_format()
%matplotlib inline 

<hr>
<font face="Calibri">

<font size="5"> <b> 1. Load Data Stack</b> </font> <img src="NotebookAddons/mapfuego.png" width="250" style="padding:5px;" align="right" /> 

<font size="3"> This notebook will use a dense 12-day repeat Sentinel-1 C-band SAR data stack. It contains imagery acquired during the year 2018 in Guatemala. The data are already prepared for you. In a later lecture, you will learn how to download and pre-process Sentinel-1 images using the services of the <a href="https://www.asf.alaska.edu/" target="_blank">Alaska Satellite Facility</a>. 

<br><br>
<font size="4"><b>2018 Volcán de Fuego eruption</b></font><br>
<font face="Calibri" size="3">On June 03, 2018, Volcán de Fuego in Guatemala erupted suddenly. 190 people lost their lives. Eruptions continued in June, and November saw a phase of renewed activity. The <b>pyroclastic flows and lahars</b> (mudflow) associated with the eruptive episodes also caused great damage to property and crops. 
<br><br>
     <center>

 <table><tr>
<td> <img src="NotebookAddons/fuegoash.jpg" style="width: 300px;"/> </td>
<td> <img src="NotebookAddons/fuegoash2.jpg" style="width: 300px;"/> </td>
</tr></table>
<font size="2">Getty Images</font>
</center>
 <br>
We will use the Sentinel-1 data stack to map land surface changes throughout this period. 
<br><br>
<font face="Calibri" size="3">Before we get started, let's first <b>create a working directory for this analysis and change into it:</b> </font>

In [ ]:
name = 'fuego'
path = f"{os.getcwd()}/{name}"
new_directory(path)
os.chdir(path)
print(f"Current working directory: {os.getcwd()}")

<font face="Calibri" size="3">We will <b>retrieve the relevant data</b> from an <a href="https://aws.amazon.com/" target="_blank">Amazon Web Service (AWS)</a> cloud storage bucket <b>using the following command</b>:</font></font>

In [ ]:
time_series_path = 's3://asf-jupyter-data/fuego.tar.gz'
time_series = os.path.basename(time_series_path)
!aws s3 cp $time_series_path $time_series

<font face="Calibri" size="3"> Now, let's <b>unzip the file (overwriting previous extractions) and clean up after ourselves:</b> </font>

In [ ]:
!tar -xvzf {name}.tar.gz

<br>
<font face="Calibri" size="5"> <b> 2. Define Some Python Helper Functions for this Notebook </b> </font> 
<br><br>
<font face="Calibri" size="3">We are defining two helper functions for this notebook:

- **CreateGeoTiff()** to write out images
- **timeseries_metrics()** to compute various metrics from a time series data stack</font>

In [ ]:
def CreateGeoTiff(Name, Array, DataType,NDV, bandnames=None, ref_image=None, 
                  GeoT=None, Projection=None):
    # If it's a 2D image we fake a third dimension:
    if len(Array.shape)==2:
        Array=np.array([Array])
    if ref_image==None and (GeoT==None or Projection==None):
        raise RuntimeWarning('ref_image or settings required.')
    if bandnames != None:
        if len(bandnames) != Array.shape[0]:
            raise RuntimeError('Need {} bandnames. {} given'
                               .format(Array.shape[0],len(bandnames)))
    else:
        bandnames=['Band {}'.format(i+1) for i in range(Array.shape[0])]
    if ref_image!= None:
        refimg=gdal.Open(ref_image)
        GeoT=refimg.GetGeoTransform()
        Projection=refimg.GetProjection()
    driver= gdal.GetDriverByName('GTIFF')
    Array[np.isnan(Array)] = NDV
    DataSet = driver.Create(Name, 
            Array.shape[2], Array.shape[1], Array.shape[0], DataType)
    DataSet.SetGeoTransform(GeoT)
    DataSet.SetProjection( Projection)
    for i, image in enumerate(Array, 1):
        DataSet.GetRasterBand(i).WriteArray( image )
        DataSet.GetRasterBand(i).SetNoDataValue(NDV)
        DataSet.SetDescription(bandnames[i-1])
    DataSet.FlushCache()
    return Name

In [ ]:
def timeseries_metrics(raster, ndv=np.nan): 
    # Make us of numpy nan functions
    # Check if type is a float array
    if not raster.dtype.name.find('float')>-1:
        raster = raster.astype(np.float32)
    # Set ndv to nan
    if not np.isnan(ndv):
        raster[np.equal(raster,ndv)] = np.nan
    # Build dictionary of the metrics
    tsmetrics={}
    rperc = np.nanpercentile(raster,[5,50,95], axis=0)
    tsmetrics['mean'] = np.nanmean(raster, axis=0)
    tsmetrics['max'] = np.nanmax(raster, axis=0)
    tsmetrics['min'] = np.nanmin(raster, axis=0)
    tsmetrics['range'] = tsmetrics['max'] - tsmetrics['min']
    tsmetrics['median'] = rperc[1]
    tsmetrics['p5'] = rperc[0]
    tsmetrics['p95'] = rperc[2]
    tsmetrics['prange'] = rperc[2]-rperc[0]
    tsmetrics['var'] = np.nanvar(raster, axis=0)
    tsmetrics['std'] = np.sqrt(tsmetrics['var'])
    tsmetrics['CV'] = np.abs(tsmetrics['var'] / tsmetrics['mean'])
    return tsmetrics

<br>
<font face="Calibri" size="5"> <b> 3. Define Data Directory and Path to VRT </b> </font> 
<br><br>
<font face="Calibri" size="3"><b>Create a variable containing the VRT filename and the image acquisition dates:</b></font>

In [ ]:
pol = 'VH' # you can also try 'VV' later
imagefile = f'stack{name}_{pol}.vrt'

<font face="Calibri" size="3"><b>Create an index of timedelta64 data with Pandas:</b></font>

In [ ]:
datefile = f'dates{name}_{pol}.csv'
tindex = pd.DatetimeIndex(open(datefile).read().split(','))

<font face="Calibri" size="3"><b>Print the bands and dates for all images in the virtual raster table (VRT):</b></font>

In [ ]:
j = 1
print(f"Bands and dates for {imagefile}")
for i in tindex:
    print("{:4d} {}".format(j, i.date()), end=' ')
    j += 1
    if j%5 == 1:
        print()

<hr>
<br>
<font face="Calibri" size="5"> <b> 4. Create a Time Series Animation to get an Idea of the Dynamics at the Site </b> </font>

<font face="Calibri" size="4"> <b> 4.1 Load Time Series Stack </b> </font>

<font face="Calibri" size="3">Now we are ready to create a time series animation from the calibrated SAR data.
<br><br>
<b>First, create a raster from band 0 and a raster stack from all the images:</b>
</font> 

In [ ]:
img = gdal.Open(imagefile)
band = img.GetRasterBand(1)
raster0 = band.ReadAsArray()
band_number = 0 # Needed for updates
rasterstack= img.ReadAsArray()

<font face="Calibri" size="3"><b>Print the bands, pixels, and lines:</b></font>

In [ ]:
print(f"Number of  bands: {img.RasterCount}")
print(f"Number of pixels: {img.RasterXSize}")
print(f"Number of  lines: {img.RasterYSize}")

<br>
<font face="Calibri" size="4"> <b> 4.2 Data Conversion between dB and Power Scales </b> </font>

The data at hand are radiometrically terrain corrected images, which are often expressed as terrain flattened $\gamma^0$ backscattering coefficients. For forest and land cover monitoring applications $\gamma^0$ is the preferred metric.
<br><br>

To use a logarithmic scale instead of the natural power scale, <b>you can set the following variable to True</b>:

In [ ]:
todB = True

labeldB = 'dB' if todB else 'linear'

def convert(raster, todB=todB):
    if todB:
        return 10 * np.log10(raster)
    else:
        return raster

<br>
<font face="Calibri" size="4"> <b> 4.3 Create Time Series Animation </b> </font>

<font face="Calibri" size="3"><b>Create and move into a directory in which to store our plots and animations:</b></font> 

In [ ]:
product_path = 'plots_and_animations'
new_directory(product_path)

<font face="Calibri" size="3">Now we can <b>create the information needed to animate our data:</b></font> 

In [ ]:
%%capture 
fig = plt.figure(figsize=(10, 5))
ax = fig.subplots()
ax.axis('off')
vmin = np.nanpercentile(convert(rasterstack), 1)
vmax = np.nanpercentile(convert(rasterstack), 99)

im = ax.imshow(convert(raster0), cmap='inferno', vmin=vmin, vmax=vmax)
cbar = fig.colorbar(im)
cbar.set_label(labeldB)
ax.set_title("{}".format(tindex[0].date()))
plt.rcParams.update({'font.size': 14})

def animate(i):
    ax.set_title("{}".format(tindex[i].date()))
    im.set_data(convert(rasterstack[i]))

# Interval is given in milliseconds
ani = animation.FuncAnimation(fig, animate, frames=rasterstack.shape[0], interval=300)

<font face="Calibri" size="3"><b>Configure matplotlib's RC settings for the animation:</b></font> 

In [ ]:
rc('animation', embed_limit=40971520.0)  # We need to increase the limit maybe to show the entire animation

<font face="Calibri" size="3"><b>Create a javascript animation of the time-series running inline in the notebook:</b></font> 

In [ ]:
HTML(ani.to_jshtml())

<font face="Calibri" size="3"><b>Delete the dummy png</b> that was saved to the current working directory while generating the javascript animation in the last code cell.</font> 

In [ ]:
try:
    os.remove('None0000000.png')
except FileNotFoundError:
    pass

<font face="Calibri" size="3"><b>Save the animation (animation.gif):</b> </font> 

In [ ]:
ani.save(os.path.join(product_path, f'animation_{labeldB}.gif'), writer='pillow', fps=2)

<br>
<div class="alert alert-success">
<font face="Calibri" size="5"> <b> <font color='rgba(200,0,0,0.2)'> <u>EXERCISE #1</u>:  </font> Can You See the Impact of the Eruption?</b> </font>

<font face="Calibri" size="3"> Can you see the impact of the ongoing eruption in the SAR images? Can you identify the lahar flows? When did they occur and how far did they reach?
</font>
</div>

<hr>
<br>
<font face="Calibri" size="5"> <b> 5. Computation and Visualization of Time Series Metrics</b> </font>

<font face="Calibri" size="3">Once a time-series was constructed, we can compute <b>a set of metrics</b> for each pixel in the stack:

- Mean 
- Median
- Maximum
- Minimum
- Range (Maximum - Minimum)
- 5th Percentile
- 95th Percentile
- PRange (95th - 5th Percentile)
- Variance
- Coefficient of Variation (Variance/Mean)

<hr>
First, we <b>mask out pixels</b> that are zero (e.g. beyond the edge of the swath). Then we <b>calculate the time series metrics</b>:
</font> 

In [ ]:
mask = (rasterstack == 0)
raster = np.ma.array(convert(rasterstack), mask=mask, dtype=np.float32)

In [ ]:
metrics = timeseries_metrics(raster.filled(np.nan), ndv=np.nan)

In [ ]:
metrics.keys()

<font face="Calibri" size="3">Let's look at the histograms for the time series variance and coeficient of variation to aid displaying those images:</font> 

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(16,4))
ax[0].hist(metrics['var'].flatten(), bins=100, range=np.nanpercentile(metrics['var'], [1,99]))
ax[1].hist(metrics['CV'].flatten(), bins=100, range=np.nanpercentile(metrics['CV'], [1,99]))
_ = ax[0].set_title('Variance')
_ = ax[1].set_title('Coefficient of Variation')

In [ ]:
# List the metrics keys you want to plot
metric_keys=['mean', 'median', 'max', 'min', 
             'p95', 'p5', 'prange', 'var', 'std', 'CV']
fig= plt.figure(figsize=(16,40))
idx=1
for i in metric_keys:
    ax = fig.add_subplot(5,2,idx)
    vmin, vmax = np.nanpercentile(metrics[i], [1, 99])
    ax.imshow(metrics[i],vmin=vmin,vmax=vmax,cmap='inferno')
    ax.set_title(i.upper())
    ax.axis('off')
    idx+=1

<hr>
<br>
<div class="alert alert-success">
<font face="Calibri" size="5"> <b> <font color='rgba(200,0,0,0.2)'> <u>EXERCISE #2:</u> </font></b> 

<font face="Calibri" size="3"> Can you group the metrics in terms of what they are most sensitive to? How would you expect the sensitivity of each metric to change if you switched from dB to linearly scaled images?
</font>
</div>
<br>
<hr>

<font face="Calibri" size="3">You might have noticed white patches in the images above. These do not contain any data. The reason is that they are in the radar shadow of terrain that is closer to the satellite.

<hr>
<br>
<font face="Calibri" size="5"> <b> 6. Some Popular SAR Change Detection Methods</b> </font>

<font face="Calibri" size="3">This section will introduce you to the following popular and simple change detection methods:

- Time series metrics  95$^{th}$ and 5$^{th}$ percentile difference and standard deviation thresholding
- Time series coefficient of variation thresholding
</font> 

<hr>
<font face="Calibri" size="4"> <b> 6.1 Change Detection with the Percentile Difference and the Variance Threshold Method</b> </font>

<font face="Calibri" size="3">In this method we find thresholds on the <b>95$^{th}$ and 5$^{th}$ percentile difference</b> or the <b>temporal pixel-by-pixel gray value cariance</b>. Let's start with the 95$^{th}$ and 5$^{th}$ percentile difference. The advantage to look at percentiles verus maximum minus minimum is that it is more robust to outliers.

First, let us define a <b>function for plotting histograms</b>
</font> 

In [ ]:
def plot_histogram_cdf(metric='std'):
    plt.rcParams.update({'font.size': 12})
    fig = plt.figure(figsize=(14, 4)) # Initialize figure with a size
    ax1 = fig.add_subplot(121)  # 121 determines: 2 rows, 2 plots, first plot
    ax2 = fig.add_subplot(122)

    h = ax1.hist(
        metrics[metric].flatten(), bins=200, range=np.nanpercentile(metrics[metric], [1, 99]))
    ax1.xaxis.set_label_text(f'{metric} {labeldB}')
    ax1.set_title('Histogram')

    n, bins, patches = ax2.hist(
        metrics[metric].flatten(), bins=200, range=np.nanpercentile(metrics[metric], [1, 99]),
        cumulative='True', density='True', histtype='step', label='Empirical')
    ax2.xaxis.set_label_text(f'{metric} {labeldB}')
    ax2.set_title('CDF')

    outind = np.where(n > 0.95)
    threshind = np.min(outind)
    thresh = bins[threshind]
    ax1.axvline(thresh,color='red')
    _ = ax2.axvline(thresh,color='red')
    plt.savefig(os.path.join(product_path, f'{metric}_{labeldB}_histogram.png'),
                dpi=200, transparent='true')

<font face="Calibri" size="3"><b>Now let's look at the 95th - 5th percentile range</b>

In [ ]:
plot_histogram_cdf(metric='prange')

<br>
<font face="Calibri" size="3">Let's visualize the 5% of all pixels with the largest (95th - 5th percentile) difference in the time series. We will refer to the pixels (x,y) that exceed this threshold $t$ as likely <b>change pixels (cp)</b>:

${cp}_{x,y} = P_{x,y}^{95th} - P_{x,y}^{5th} > t$ 

If we define $t$ to correspond to the 5% of pixels with highest (95th - 5th percentile) difference, the image looks like:</font> 

In [ ]:
def plot_threshold_classifier(metric='prange', percentage_cutoff=5):
    plt.figure(figsize=(8,8))
    thresh = np.nanpercentile(metrics[metric], 100 - percentage_cutoff)
    mask = metrics[metric] < thresh # For display we prepare the inverse mask
    plt.imshow(mask, cmap='gray')
    _=plt.title(f'Threshold Classifier on {metric} > %1.3f' % thresh)
    plt.savefig(os.path.join(product_path, f'changes_{metric}_{labeldB}.png'),
                dpi=200, transparent='true')
    return np.logical_not(mask)

metric = 'prange'
masks = {metric: plot_threshold_classifier(metric=metric)}

<hr>
<br>
<div class="alert alert-success">
<font face="Calibri" size="5"> <b> <font color='rgba(200,0,0,0.2)'> <u>EXERCISE #3:</u> </font></b> 

<font face="Calibri" size="3"> Discuss what you see in this figure. What are the black areas? What are the white areas? What kind of changes may be included in this map?
</font>
</div>
<br>
<hr>

<br>
<font face="Calibri" size="3">Instead of applying a threshold on the 95th - 5th percentile difference data, we can also attempt to threshold other metrics. The <b>standard deviation</b> (or variance) variable seems a useful indicator for change as it identifies pixels for which radar brightness has changed strongly within the time series. Hence, in the following we use this metric for change identification according to:

${cp}_{x,y} = \sigma > t$ 

with $t=CDF_{\sigma} > 0.95$ (5% pixels with highest standard deviation):</font> 

In [ ]:
plot_histogram_cdf(metric='std')

In [ ]:
metric = 'std'
masks[metric] = plot_threshold_classifier(metric=metric)

<br>
<font face="Calibri" size="4"> <b> 6.2 Change Detection with the Coefficient of Variation Method </b> </font>

<font face="Calibri" size="3">We can also set a threshold $t$ for the <b>coefficient of variation image</b>
to classify change in the time series:
    
${CV}_{x,y} = \frac{\sigma_{x,y}}{\overline{X}_{x,y}} > t$ 

Let's look at the histogram and the Cumulative Distribution Function (CDF) of the coefficient of variation:</font> 

In [ ]:
plot_histogram_cdf(metric='CV')

<br>
<font face="Calibri" size="3">With a threshold of $t=CDF_{CV} > 0.95$ (5% pixels with highest variance) the change pixels would look like the following image:</font> 

In [ ]:
metric = 'CV'
masks[metric] = plot_threshold_classifier(metric=metric)

<br>
<font face="Calibri" size="4"> <b> 6.3 Pair-wise change detection </b> </font>

<font face="Calibri" size="3">To analyze temporal changes between two images, it is useful to compute metrics that are sensitive to discrepancies between the two images. In radar remote sensing, the standard way is to look at ratios (in the linearly scaled power domain) or, equivalently, at differences in the logarithmic dB domain. 

In [ ]:
dates = ('2018-05-27', '2018-06-08') # around first eruption

# convert to datetime objects
dates_ = [datetime.datetime.strptime(date, '%Y-%m-%d') for date in dates]

# get the indices in one line
dates_ind = [np.argmin(np.abs(date - tindex)) for date in dates_]
print(f'Comparing image {dates_ind[0]} from {tindex[dates_ind[0]].date()} with {dates_ind[1]} from {tindex[dates_ind[1]].date()}')

<br>
<font face="Calibri" size="3"><b>Compute the log ratio in dB</b>, corresponding to the difference in dB</font>

In [ ]:
ratiodB = 10 * np.log10(rasterstack[dates_ind[1], ...] / rasterstack[dates_ind[0], ...])

<br>
<font face="Calibri" size="3"><b>Let us manually choose a threshold this time</b>
<br><br>
thresh is the threshold, e.g. -2 dB<br>
thresh_type determines whether we mask everything below that (lower) or above (upper)</font>

In [ ]:
thresh = -2
thresh_type = 'lower' #'lower': mask everything below thresh, 'upper': mask everything above

dynamic_range = np.nanpercentile(np.abs(ratiodB), 99)
fig, axs = plt.subplots(ncols=3, nrows=1)
fig.suptitle(f'{tindex[dates_ind[0]].date()} to {tindex[dates_ind[-1]].date()}     ', ha='right', fontsize='15')
fig.set_size_inches(20, 4)
plt.subplots_adjust(hspace=0.4, right=0.85)
h = axs[0].hist(
        ratiodB.flatten(), bins=200, range=np.nanpercentile(ratiodB, [0.1, 99.9]))
axs[0].xaxis.set_label_text(f'difference [dB]')
axs[0].set_title('Histogram')
mask = (ratiodB > thresh if thresh_type == 'lower' else ratiodB < thresh).astype(np.int8)
axs[1].imshow(mask, cmap='gray')
axs[1].set_title('Mask')
im0 = axs[2].imshow(ratiodB, cmap='RdBu', vmin=-dynamic_range, vmax=dynamic_range)
cbar = fig.colorbar(im0, orientation='vertical', ax=axs.ravel().tolist())
cbar.set_label('[dB]')
axs[2].set_title('Image')
logratiolabel = f'logratio_{tindex[dates_ind[0]].date()}_{tindex[dates_ind[0]].date()}'
plt.savefig(os.path.join(product_path, f'{logratiolabel}.png'),
            dpi=200, transparent='true')
masks[logratiolabel] = np.logical_not(mask)

<br>
<font face="Calibri" size="3"><b>Do you think the threshold is appropriate? </b></font>

<br>
<div class="alert alert-success">
<font face="Calibri" size="5"> <b> <font color='rgba(200,0,0,0.2)'> <u>EXERCISE #4:</u> </font></b> 

<font face="Calibri" size="3"> Discuss what you see in this figure: 
- What kind of changes are detected? 
- How does this change map compare to the previous one? Note that you are only looking at one specific pair of images here.
- Feel free to look at other image pairs and analyze identified changes!
</font>
</div>
<br>

<hr>
<br>
<font face="Calibri" size="5"> <b> 7. Write Our Change Detection Results and Metrics Images to GeoTIFF files</b> </font>

<font face="Calibri" size="4"> <b> 7.1 Determine Output Geometry </b> </font>

<font face="Calibri" size="3">First, we need to <b>set the correct geotransformation and projection information</b>. We retrieve the values from the input images:
</font> 

In [ ]:
proj = img.GetProjection()
geotrans = list(img.GetGeoTransform())
geotrans

<br>
<font face="Calibri" size="4"> <b> 7.2 Output Time Series Metrics Images </b> </font>

<font face="Calibri" size="3">We use the root of the time series data stack name and append a _ts_metrics_&lt;metric&gt;.tif ending as filenames:
</font> 

In [ ]:
# Time Series Metrics as image image:
# We make a new subdirectory where we will store the images
dirname = f'{name}_tsmetrics'
os.makedirs(dirname,exist_ok=True)
print(dirname)

<br>
<font face="Calibri" size="3">Now we can <b>output the individual metrics as GeoTIFF images</b>:
</font> 

In [ ]:
names=[] # List to keep track of all the names
for metric in metrics:
    name_ = os.path.join(dirname, f'{metric}_{labeldB}.tif')
    CreateGeoTiff(name_,metrics[metric], gdal.GDT_Float32, np.nan,[metric],
                  GeoT=geotrans, Projection=proj)
    names.append(name_)

<br>
<font face="Calibri" size="4"> <b> 7.3 Build a Virtual Raster Table on the Metrics GeoTIFF images </b> </font>

<font face="Calibri" size="3">To tie the images into one new raster stack of time series metrics we build a virtual raster table with all the metrics. 

Trick: Use ' '.join(names) to build one long string of names separated by a space as input to *gdalbuildvrt*:
</font> 

In [ ]:
cmd='gdalbuildvrt -separate -overwrite -vrtnodata nan '+\
   dirname+f'_{labeldB}.vrt '+' '.join(names)
# print(cmd)
_ = os.system(cmd)

<br>
<font face="Calibri" size="4"> <b> 7.4 Create GeoTIFFs for the Change Images from our Four Change Detection Attempts </b> </font>

<font face="Calibri" size="3">We are going to write GeoTIFF output files that store the results from the classifiers:
</font> 

In [ ]:
for metric in masks:
    fnmetric = os.path.join(dirname, f'{name}_{labeldB}_{metric}_thresholds.tif')
    CreateGeoTiff(fnmetric, masks[metric], gdal.GDT_Byte, np.nan, 
                  GeoT=geotrans,Projection=proj)


<hr>
<br>
<div class="alert alert-success">
<font face="Calibri" size="5"> <b> <font color='rgba(200,0,0,0.2)'> <u>EXERCISE #5:</u> </font></b> 

<font face="Calibri" size="3"> Explore the data stack a bit more. Answer the following questions for yourself:

<ul>
  <li> Which metrics are more appropriate for logarithic (dB) or linearly (power) transformed images?</li>
  <li> Change the thresholds and dates in this notebook to see the effects on detected changes.</li>
  <li> Load created change masks into QGIS and compare the detected areas with your time series plots and image data.</li>
</ul>

</font>
</div>
<br>
<hr>

<div class="alert alert-success">
<font face="Calibri" size="5"> <b> <font color='rgba(200,0,0,0.2)'> <u>EXERCISE #6:</u> </font></b> 

<font face="Calibri" size="4"><b>2016-2017 flooding</b></font>
<font face="Calibri" size="3"><br>Explore the flooding data stack from Lab 3 at VV and VH polarization. What can you detect? Compare the minimum method for flood detection from Lab 3 with the results obtained with this notebook.
<br><br>
To this end, we advise you to restart the notebook (or duplicate it first) by going to 'Kernel' and 'Restart'
</font>
</div>
<br>
<hr>

<font face="Calibri" size="2"> <i>SAR Training Materials - Version 1.2 - September 2020
    <br>
    <b>Version Changes</b>
    <ul>
        <li>Handle osl_dark theme</li>
    </ul>
        </i>
</font>